In [1]:
import torch
from torch.utils.data import DataLoader,TensorDataset
import torch.nn as nn
dtype=torch.FloatTensor #32位

1、建索引表

1.1句子拆成单词

In [2]:
sentences=['i like cat', 'i love coffee', 'i hate milk']
# sentences=[i.split() for i in sentences] #.split只能对str用# sentences=''.join('ab c').split()# sentences=''.join(sentences) #句子之间就没加空格，有空格不会再加空格# sentences.split()
sentences_list=' '.join(sentences).split() #.join .split是str操作 默认通过空格分开单词
print(sentences_list)


['i', 'like', 'cat', 'i', 'love', 'coffee', 'i', 'hate', 'milk']


1.2 建词单词的索引表 用字典

In [3]:
vocab=list(set(sentences_list)) #去重用set 因为集合无重复元素
word2idx={w:i for i,w in enumerate(vocab)} #妙啊enumerate
idx2word={i:w for i,w in enumerate(vocab)}
V=len(vocab)
m=2 #词向量表的每一个单词的表示


2 准备X,y 转成数字

In [4]:
def makedata(sentences):
    input,output=[],[]
    for sen in sentences:
        sen=sen.split()
        input_tmp=[word2idx[w] for w in sen]
        output_tmp=word2idx[sen[-1]]

        input.append(input_tmp)
        output.append(output_tmp)
    return input,output


打包

In [5]:
input,output=makedata(sentences)
input,output=torch.LongTensor(input),torch.LongTensor(output)
dataset=TensorDataset(input,output)
loader=DataLoader(dataset,2,True) 

3 网络

In [10]:
m=2
n_step=2
n_hidden=10
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM,self).__init__()
        self.C=nn.Embedding(V,m)
        self.H=nn.Parameter(torch.randn(n_step*m,n_hidden).type(dtype))
        self.d=nn.Parameter(torch.randn(n_hidden).type(dtype))
        self.U=nn.Parameter(torch.randn(n_hidden,V).type(dtype))
        self.b=nn.Parameter(torch.randn(V).type(dtype)) 
        self.W=nn.Parameter(torch.randn(n_step*m,V).type(dtype))      
    
    def forward(self,X):
        X=self.C(X)
        # print(X.shape)
        X=X.view(-1,n_step*m)
        hidden_out=torch.tanh(torch.mm(X,self.H)+self.d)
        output=torch.mm(X,self.W)+torch.mm(hidden_out,self.U)+self.b
        # output=torch.softmax(output) 
        return output



4 train

In [11]:
model=NNLM()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3)
criterion=nn.CrossEntropyLoss()
for epoch in range(100):
    for batch_x,batch_y in loader:
        pred=model(batch_x)
        loss=criterion(pred,batch_y)
        
        
        if (epoch+1)%100==0:
            print(epoch+1,loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

ValueError: Expected input batch_size (3) to match target batch_size (2).

5 test

In [ ]:
pred=model(input)
print(pred)
pred_idx=pred.max(1,keepdim=True)[1]
print(pred_idx)
print([idx2word[idx] for idx in pred_idx.squeeeze()])


RuntimeError: shape '[-1, 4]' is invalid for input of size 18